# HW4 Chenyu Wang

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import least_squares, leastsq

# Problem 1

In [2]:
df = pd.read_csv('CMT_Rates.csv',index_col = 'Date')
df.head()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
11/1/3,0.11,0.15,0.19,0.29,0.61,1.03,2.02,2.74,3.36,4.18,4.39
11/1/4,0.12,0.14,0.19,0.28,0.63,1.04,2.01,2.72,3.36,4.21,4.44
11/1/5,0.13,0.14,0.19,0.31,0.71,1.16,2.14,2.86,3.50,4.34,4.55
11/1/6,0.13,0.15,0.18,0.30,0.68,1.11,2.09,2.80,3.44,4.31,4.53
11/1/7,0.13,0.14,0.18,0.29,0.60,1.02,1.96,2.69,3.34,4.25,4.48


In [3]:
sample = df.loc['11/1/3':'15/12/31']
sample.tail()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
15/12/24,0.15,0.20,0.49,0.64,1.03,1.33,1.73,2.06,2.25,2.61,2.96
15/12/28,0.13,0.23,0.51,0.66,1.05,1.33,1.73,2.05,2.24,2.59,2.95
15/12/29,0.18,0.23,0.50,0.67,1.09,1.38,1.81,2.12,2.32,2.69,3.04
15/12/30,0.08,0.21,0.47,0.64,1.08,1.36,1.80,2.14,2.31,2.69,3.04
15/12/31,0.14,0.16,0.49,0.65,1.06,1.31,1.76,2.09,2.27,2.67,3.01


## Step 1: Assume a value of $\lambda$, and $\beta t 1(t)$; $\beta 2(t)$; $\beta 3(t)$ to a set of yields observed on day t (repeat for all days in the dataset)

## Step 2: Find $\lambda$ that bests fits the whole dataset (Step1 needs to be repeated on each iteration; as you search for optimal  $\lambda$). You can use an optimization package, or write your own code

## Method 1 : Function

In [4]:
def New_loading(lambda_):
    loading_1 = np.vstack([1]*len(time))
    loading_2 = np.vstack((1-np.exp(-lambda_ * tao)) / (lambda_ * tao) for tao in time)
    loading_3 = np.vstack((1-np.exp(-lambda_ * tao)) / (lambda_ * tao) - np.exp(-lambda_ * tao) for tao in time)
    loading = np.hstack([loading_1, loading_2, loading_3])
    return loading

In [5]:
def residual(lambda_):
    residuals = []
    for i in range(sample.shape[0]):
        y = np.array(sample.loc[sample.index[i]])
        model = sm.OLS(y, New_loading(lambda_))
        result = model.fit()
        y_pred = result.predict(New_loading(lambda_))
        residual = y - y_pred
        residuals.append(residual)
    a = np.concatenate(residuals)
    error_sq_sum = (a.T @ a) / len(a)
    return error_sq_sum

In [6]:
def Optimization(fun, lambda_0):
    return least_squares(fun, lambda_0, method = 'lm', ftol = 1e-12)

In [7]:
### Initialize terms
time = [1/12,3/12,6/12,1,2,3,5,7,10,20,30]
### Initialize lambda (using the number in paper)
lambda_ = 0.0609

In [8]:
print('The optimal lambda is :',Optimization(residual, lambda_).x)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


The optimal lambda is : [0.47908436]


## Method 2 : Class

In [9]:
### Short rate model
class NDS:
    ### Initial data and parameters
    def __init__(self, data, lambda_, time):
        self.data = data
        self.lambda_ = lambda_
        self.time = time
        self.loading = self.new_loading()
        self.betas = []
    
    ### Update parameters
    def update_betas(self, lambda_):
        self.lambda_ = lambda_
        self.betas = []
        self.loading = self.new_loading()
    
    ### Calculate loading in case of changing of lambda
    def new_loading(self):
        loading_1 = np.vstack([1]*len(self.time))
        loading_2 = np.vstack((1-np.exp(-self.lambda_ * tao)) / (self.lambda_ * tao) for tao in self.time)
        loading_3 = np.vstack((1-np.exp(-self.lambda_ * tao)) / (self.lambda_ * tao) - np.exp(-self.lambda_ * tao) for tao in self.time)
        loading = np.hstack([loading_1, loading_2, loading_3])
        return loading
    
    ### Calculate residuals and prepare for least square optimizor
    def residual(self, lambda_):
        ### Update new lambda and loadings
        self.update_betas(lambda_)
        ### Initialize residual vector
        residuals = []
        ### for loop and calculate all the parameters and get new residuals
        for i in range(self.data.shape[0]):
            ### Get y
            y = np.array(self.data.loc[self.data.index[i]])
            ### OLS
            model = sm.OLS(y, self.loading).fit()
            ### Get y_hat
            y_pred = model.predict(self.loading)
            ### Store betas
            self.betas.append(model.params)
            ### Store residuals
            residuals.append(y - y_pred)
        ### Return residual vector
        a = np.concatenate(residuals)
        ### Calculate error_square_sum
        error_square_sum = (a.T @ a)/len(a)
        return error_square_sum
    
    ### Use optimizor in scipy to get the optimal lambda
    def optimization(self, lambda_0):
        return least_squares(self.residual, lambda_0, method = 'lm', ftol = 1e-12)
        

In [10]:
### Initialize NDS object
NDSmodel = NDS(sample, lambda_, time)
Op_lambda = NDSmodel.optimization(lambda_)
print('The optimal lamnbda is :',Op_lambda.x)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


The optimal lamnbda is : [0.47908436]


### Therefore, both of the two methods provide the same result. The optimal $\lambda$ is 0.479

# Problem 2

## Step 1: Compute RMSE for each day in the sample and pick 20 days with largest RMSE

In [11]:
### Compute RMSE
def Compute_RMSE(residual):
    RMSE = np.sqrt((residual.T @ residual).sum() / len(residual))
    return RMSE

In [12]:
### Extract all residual data
def residual_all(lambda_):
    residuals = []
    for i in range(sample.shape[0]):
        y = np.array(sample.loc[sample.index[i]])
        model = sm.OLS(y, New_loading(lambda_))
        result = model.fit()
        y_pred = result.predict(New_loading(lambda_))
        residual = y - y_pred
        residuals.append(residual)
    return np.concatenate(residuals)
residual_df = pd.DataFrame(residual_all(0.479).reshape(-1,len(time)), index = sample.index, columns = sample.columns)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


In [13]:
residual_df.head()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
11/1/3,0.036496,0.042680,0.019754,-0.040044,-0.118059,-0.129032,0.064569,0.154928,0.117706,-0.012411,-0.136586
11/1/4,0.040401,0.029019,0.019155,-0.046302,-0.090705,-0.112096,0.055846,0.129368,0.103743,-0.009723,-0.118706
11/1/5,0.064244,0.029615,0.001913,-0.062921,-0.097091,-0.101466,0.056675,0.135496,0.110843,-0.005649,-0.131658
11/1/6,0.053039,0.035476,-0.002849,-0.052681,-0.087972,-0.103015,0.059823,0.126295,0.095937,-0.001885,-0.122167
11/1/7,0.034146,0.020586,0.010836,-0.019323,-0.087401,-0.094209,0.037418,0.118805,0.086679,0.005726,-0.113263


In [14]:
### Calculate RMSE
RMSE_day = []
for date in residual_df.index:
    RMSE_day.append(Compute_RMSE(np.array(residual_df.loc[date])))
RMSE = pd.DataFrame(RMSE_day, index = sample.index, columns = ['RMSE'])

In [15]:
### Get the date with largest RMSE
RMSE_max = RMSE.sort_values(by = ['RMSE'], ascending = 0).iloc[:20]
RMSE_max

,RMSE
Date,
13/12/31,0.116172
14/4/24,0.112497
13/12/24,0.112037
14/4/21,0.111857
13/12/27,0.111262
14/4/17,0.110826
14/4/22,0.110645
14/1/2,0.110439
12/12/6,0.110314


## Step 2: Use 6m of data prior to each of the 20 days to fit AR(1) to each of $\beta_1(t)$; $\beta_2(t)$; $\beta_3(t)$ time series and compute half-lives (HL). Generate t+5d. t+10d, t+1m forecast for each of the 20 days, using the corresponding model for each forecast. Total of 3 forecasts for each of the 20 days.

In [16]:
### Extract beta
Beta_all = pd.DataFrame(np.vstack(NDSmodel.betas), index = sample.index, columns = ['beta1', 'beta2', 'beta3'])

### Train set

In [17]:
### Extract 20 days and 6 month data prior to each of the 20 days
def Get_6mon(date, data):
    end = data.index.get_loc(date)
    start = end - 6*22
    return data.iloc[start:end,:]

### Initial a list to store 20 dataframes
data_prior = []
for date in RMSE_max.index:
    data_prior.append(Get_6mon(date,Beta_all))

### Test set

In [18]:
### Extract test_set
def Get_test(date, data, period):
    start = data.index.get_loc(date) + 1
    end = start + period
    return data.iloc[start:end,:]

In [19]:
### Extract 5 day forward
test_5day = []
for date in RMSE_max.index:
    test_5day.append(Get_test(date, sample, 5))
### Extract 10 day forward
test_10day = []
for date in RMSE_max.index:
    test_10day.append(Get_test(date, sample, 10))
### Extract 20 day forward
test_20day = []
for date in RMSE_max.index:
    test_20day.append(Get_test(date, sample, 20))

In [20]:
len(test_10day)

20

### Fit AR(1) forecast Beta

In [21]:
from statsmodels.tsa.arima_model import ARMA
### Fit AR(1) model
def AR_1(Series, n):
    model = ARMA(Series, (1,0))
    result = model.fit()
    forecast = result.forecast(steps = n)[0]
    
    return forecast

In [22]:
### Forecast beta 1
beta1_5day = []
beta1_10day = []
beta1_20day = []
for i in range(len(data_prior)):
    beta1_5day.append(AR_1(data_prior[i]['beta1'], 5))
    beta1_10day.append(AR_1(data_prior[i]['beta1'], 10))
    beta1_20day.append(AR_1(data_prior[i]['beta1'], 20))
### Forecast beta 2
beta2_5day = []
beta2_10day = []
beta2_20day = []
for i in range(len(data_prior)):
    beta2_5day.append(AR_1(data_prior[i]['beta2'], 5))
    beta2_10day.append(AR_1(data_prior[i]['beta2'], 10))
    beta2_20day.append(AR_1(data_prior[i]['beta2'], 20))
### Forecast beta 3
beta3_5day = []
beta3_10day = []
beta3_20day = []
for i in range(len(data_prior)):
    beta3_5day.append(AR_1(data_prior[i]['beta3'], 5))
    beta3_10day.append(AR_1(data_prior[i]['beta3'], 10))
    beta3_20day.append(AR_1(data_prior[i]['beta3'], 20))

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

### Compute half-life

In [23]:
### Compute half-life
def Half_life(Series):
    ### Get y_t
    ts_shift = np.array(Series.drop(Series.index[0]))
    ### Get y_t-1
    ts_back = Series.drop(Series.index[len(Series)-1])
    ### Add constant
    ts_back = np.array(sm.add_constant(ts_back))
    ### Fit AR(1)
    model = sm.OLS(ts_shift, ts_back)
    result = model.fit()
    ### compute half-life
    half_life = - np.log(2) / np.log(abs(result.params[1]))
    
    return half_life

In [24]:
### Compute half_life
Beta1_HL = []
Beta2_HL = []
Beta3_HL = []
for i in range(len(data_prior)):
    Beta1_HL.append(Half_life(data_prior[i]['beta1']))
    Beta2_HL.append(Half_life(data_prior[i]['beta2']))
    Beta3_HL.append(Half_life(data_prior[i]['beta3']))
### Store in dataframe
HL_dic = {
    'Beta1':Beta1_HL,
    'Beta2':Beta2_HL,
    'Beta3':Beta3_HL
}
HL_df = pd.DataFrame(HL_dic, index = RMSE_max.index)
HL_df

/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,Beta1,Beta2,Beta3
Date,,,
13/12/31,9.156089,8.747117,16.323831
14/4/24,31.099103,15.559185,243.457569
13/12/24,7.410909,6.860290,15.608008
14/4/21,24.414817,13.566093,340.820384
13/12/27,7.646834,6.948104,16.021393
14/4/17,32.042925,16.266062,126.396909
14/4/22,24.603481,13.255378,183.319898
14/1/2,10.912630,10.964342,15.230329
12/12/6,9.314572,7.919010,21.038748


### Get loading

In [25]:
loading = New_loading(0.479)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


### Get prediction

In [26]:
def Get_prediction(beta1, beta2, beta3, loading, n):
    pred_n_day = []
    for i in range(len(RMSE_max.index)):
        beta1_ = beta1[i]
        beta2_ = beta2[i]
        beta3_ = beta3[i]
        prediction = {}
        for j in range(n):
            beta_vec = [beta1_[j], beta2_[j], beta3_[j]]
            name = 'day_' + str(j+1)
            prediction[name] = loading @ np.array(beta_vec)
        pred_n_day.append(pd.DataFrame(prediction, index = sample.columns).T)
    return pred_n_day        

In [27]:
pred_5_day = Get_prediction(beta1_5day, beta2_5day, beta3_5day, loading, 5)
pred_10_day = Get_prediction(beta1_10day, beta2_10day, beta3_10day, loading, 10)
pred_20_day = Get_prediction(beta1_20day, beta2_20day, beta3_20day, loading, 20)

In [28]:
pred_5_day[0]

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
day_1,-0.010555,0.011514,0.057337,0.184826,0.525691,0.908802,1.632589,2.212472,2.821861,3.706718,4.018304
day_2,-0.008072,0.013139,0.057791,0.183291,0.521276,0.902516,1.624253,2.203197,2.811969,3.696281,4.007707
day_3,-0.005747,0.014650,0.058186,0.181796,0.517044,0.896507,1.616298,2.194350,2.802539,3.686337,3.997613
day_4,-0.003569,0.016052,0.058529,0.180340,0.512988,0.890762,1.608706,2.185912,2.793550,3.676863,3.987997
day_5,-0.001529,0.017354,0.058822,0.178925,0.509100,0.885269,1.601459,2.177865,2.784980,3.667836,3.978837


### Result explanation

Until now, we have already finished prediction of 5, 10, 20 days. These results are stored in pred_5_day, pred_10_day, pred_20_day. In each list, there are 20 dataframes, each dataframen is a n-day prediction of the term structure. For example, the first dataframe represents the predition of the forward n-day of the day with maximum squared error. Therefore, there are total 60 dataframs.

In [29]:
def Get_day_pred(df_list, n):
    pred = []
    for i in range(len(df_list)):
        pred.append(df_list[i].iloc[-1,:])
    df = pd.concat(pred, axis = 1).T
    df.index = RMSE_max.index
    return df

#### Get t+5 prediction

In [30]:
Get_day_pred(pred_5_day, 5)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,-0.001529,0.017354,0.058822,0.178925,0.509100,0.885269,1.601459,2.177865,2.784980,3.667836,3.978837
14/4/24,-0.083933,-0.033388,0.048039,0.225318,0.604970,0.978853,1.627740,2.120457,2.623862,3.341291,3.592661
13/12/24,-0.008827,0.010174,0.051646,0.171255,0.499149,0.872215,1.581956,2.152917,2.754164,3.628360,3.936299
14/4/21,-0.073606,-0.024599,0.054999,0.230042,0.609183,0.985553,1.642468,2.143191,2.655842,3.387481,3.643928
13/12/27,-0.019501,0.003734,0.051035,0.180664,0.523476,0.906661,1.628288,2.205354,2.811206,3.690387,3.999923
14/4/17,-0.065662,-0.024038,0.045599,0.204174,0.560812,0.923946,1.568970,2.066363,2.578799,3.313214,3.570925
14/4/22,-0.064694,-0.017186,0.060426,0.232300,0.607479,0.981922,1.637945,2.139252,2.653204,3.387381,3.644781
14/1/2,-0.003878,0.015941,0.058878,0.182066,0.518581,0.900805,1.627272,2.211360,2.826258,3.720140,4.034999
12/12/6,0.198429,0.148076,0.092810,0.042284,0.104862,0.285878,0.748276,1.175647,1.654900,2.379158,2.636828


#### Get t+10 prediction

In [31]:
Get_day_pred(pred_10_day, 10)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,0.006862,0.022566,0.059689,0.172427,0.491956,0.861227,1.569894,2.142877,2.747775,3.628710,3.939155
14/4/24,-0.083211,-0.033099,0.047892,0.224928,0.605786,0.982065,1.636608,2.134392,2.643405,3.369243,3.623598
13/12/24,-0.004724,0.011066,0.048218,0.160722,0.479029,0.846591,1.551665,2.121597,2.723197,3.599257,3.907978
14/4/21,-0.068147,-0.020082,0.058386,0.232015,0.610684,0.988378,1.649809,2.155103,2.673063,3.412886,3.672259
13/12/27,-0.014176,0.005268,0.047454,0.168617,0.499837,0.876178,1.591602,2.166878,2.772533,3.653013,3.963153
14/4/17,-0.069371,-0.028189,0.041025,0.199457,0.557699,0.923746,1.575481,2.078822,2.597810,3.342025,3.603214
14/4/22,-0.058140,-0.011615,0.064806,0.235146,0.609636,0.985215,1.645462,2.151135,2.670197,3.412281,3.672512
14/1/2,0.005349,0.022111,0.060820,0.176693,0.502163,0.876741,1.593948,2.173069,2.784044,3.673453,3.986849
12/12/6,0.195799,0.146568,0.092796,0.044670,0.110307,0.292881,0.756173,1.183294,1.661773,2.384424,2.641484


#### Get t+20 prediction

In [32]:
Get_day_pred(pred_20_day, 20)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,0.017097,0.028404,0.059519,0.162084,0.466942,0.826735,1.525129,2.093486,2.695433,3.573880,3.883616
14/4/24,-0.074520,-0.025876,0.053375,0.228300,0.608748,0.987496,1.649893,2.155475,2.673480,3.413127,3.672415
13/12/24,0.002617,0.013645,0.044368,0.146231,0.449959,0.808903,1.506147,2.073799,2.675118,3.552761,3.862224
14/4/21,-0.054623,-0.008811,0.066844,0.236553,0.612223,0.990734,1.658267,2.170604,2.697105,3.450404,3.714621
13/12/27,-0.004843,0.008976,0.043549,0.152056,0.465753,0.831537,1.536954,2.108911,2.713567,3.594935,3.905606
14/4/17,-0.067109,-0.027426,0.040040,0.196467,0.554818,0.924051,1.585138,2.097548,2.626900,3.386948,3.653785
14/4/22,-0.043960,0.000301,0.073888,0.240255,0.611611,0.987862,1.653938,2.166437,2.693813,3.449045,3.714005
14/1/2,0.016151,0.028715,0.061627,0.167483,0.477873,0.842056,1.546754,2.119222,2.724990,3.608517,3.919999
12/12/6,0.193611,0.145955,0.094268,0.049465,0.119238,0.303804,0.767967,1.194420,1.671473,2.391371,2.647399


## Step 3 : Do these forecasts perform better than those generated under a random walk assumption?

## Random walk

In [33]:
def RW_pred(time_list, n, data):
    pred_list = []
    for date in time_list:
        start = data.index.get_loc(date)
        end = start + n
        pred_list.append(data.iloc[start:end,:])
    return pred_list

In [34]:
RW_5_day = RW_pred(RMSE_max.index, 5, sample)
RW_10_day = RW_pred(RMSE_max.index, 10, sample)
RW_20_day = RW_pred(RMSE_max.index, 20, sample)

#### Get t+5 prediction

In [35]:
Get_day_pred(RW_5_day, 5)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,0.01,0.04,0.08,0.13,0.40,0.80,1.69,2.37,2.96,3.64,3.88
14/4/24,0.02,0.03,0.05,0.11,0.42,0.87,1.69,2.25,2.67,3.22,3.47
13/12/24,0.01,0.07,0.10,0.13,0.38,0.78,1.75,2.45,3.04,3.72,3.96
14/4/21,0.01,0.03,0.04,0.11,0.43,0.90,1.72,2.28,2.68,3.20,3.45
13/12/27,0.02,0.07,0.10,0.13,0.41,0.80,1.73,2.42,3.01,3.69,3.93
14/4/17,0.00,0.01,0.04,0.10,0.43,0.91,1.74,2.30,2.70,3.22,3.46
14/4/22,0.00,0.03,0.05,0.10,0.44,0.90,1.73,2.29,2.70,3.23,3.47
14/1/2,0.00,0.05,0.08,0.13,0.43,0.87,1.77,2.44,3.01,3.67,3.90
12/12/6,0.04,0.07,0.10,0.14,0.25,0.32,0.66,1.11,1.72,2.48,2.90


#### Get t+10 prediction

In [36]:
Get_day_pred(RW_10_day, 10)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,0.00,0.04,0.06,0.11,0.39,0.78,1.65,2.30,2.88,3.54,3.80
14/4/24,0.03,0.03,0.05,0.10,0.41,0.89,1.65,2.18,2.62,3.12,3.40
13/12/24,0.00,0.05,0.08,0.13,0.43,0.87,1.77,2.44,3.01,3.67,3.90
14/4/21,0.01,0.02,0.05,0.10,0.42,0.89,1.67,2.20,2.60,3.12,3.37
13/12/27,0.01,0.05,0.06,0.12,0.39,0.77,1.64,2.29,2.88,3.54,3.80
14/4/17,0.02,0.03,0.05,0.10,0.41,0.86,1.66,2.21,2.63,3.16,3.41
14/4/22,0.01,0.03,0.05,0.11,0.43,0.90,1.68,2.22,2.63,3.16,3.41
14/1/2,0.01,0.04,0.06,0.13,0.41,0.81,1.68,2.33,2.90,3.55,3.81
12/12/6,0.03,0.05,0.10,0.15,0.28,0.39,0.77,1.24,1.82,2.58,2.99


#### Get t+20 prediction

In [37]:
Get_day_pred(RW_20_day, 20)

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
13/12/31,0.04,0.04,0.07,0.11,0.36,0.71,1.52,2.15,2.69,3.36,3.62
14/4/24,0.04,0.04,0.06,0.09,0.37,0.79,1.55,2.08,2.54,3.13,3.42
13/12/24,0.01,0.04,0.05,0.11,0.39,0.77,1.62,2.24,2.79,3.44,3.68
14/4/21,0.01,0.03,0.05,0.09,0.38,0.80,1.56,2.08,2.52,3.07,3.34
13/12/27,0.04,0.05,0.07,0.11,0.37,0.76,1.61,2.24,2.78,3.43,3.67
14/4/17,0.01,0.03,0.05,0.09,0.38,0.80,1.55,2.05,2.50,3.05,3.33
14/4/22,0.01,0.03,0.05,0.09,0.36,0.79,1.56,2.09,2.54,3.11,3.39
14/1/2,0.04,0.02,0.06,0.10,0.36,0.72,1.55,2.19,2.72,3.40,3.65
12/12/6,0.06,0.07,0.11,0.15,0.27,0.41,0.82,1.32,1.93,2.70,3.10


## Compute RMSE

### Step 1 : Create Function

In [38]:
def Get_array(df_list, n, term):
    array_list = []
    for i in range(n):
        array_list.append(np.array(df_list[i][term]))
    return np.concatenate(array_list)

In [39]:
def Cal_RMSE(AR, RW, test, term, n):
    AR_array = Get_array(AR, n, term)
    RW_array = Get_array(RW, n, term)
    test_array = Get_array(test, n, term)
    residual_AR = AR_array - test_array
    residual_RW = RW_array - test_array
    RMSE_AR = np.sqrt((residual_AR.T @ residual_AR) / len(residual_AR))
    RMSE_RW = np.sqrt((residual_RW.T @ residual_RW) / len(residual_RW))
    return [RMSE_AR, RMSE_RW]

In [40]:
def Compare_RMSE(AR, RW, test, term_list, n):
    RMSE = {}
    for date in term_list:
        RMSE[str(date)] = Cal_RMSE(AR, RW, test, date, n)
    name1 = 'AR_'+str(n)+'day'
    name2 = 'RW_'+str(n)+'day'
    df = pd.DataFrame(RMSE, index = [name1, name2], columns = term_list)
    return df

In [41]:
df1 = Compare_RMSE(pred_5_day, RW_5_day, test_5day, sample.columns, 5)
df2 = Compare_RMSE(pred_10_day, RW_10_day, test_10day, sample.columns, 10)
df3 = Compare_RMSE(pred_20_day, RW_20_day, test_20day, sample.columns, 20)
df = pd.concat([df1,df2,df3])

In [42]:
df

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
AR_5day,0.061082,0.056068,0.029586,0.087772,0.146220,0.106267,0.104708,0.197821,0.163958,0.092502,0.110599
RW_5day,0.008718,0.009592,0.008718,0.007483,0.016248,0.024331,0.029799,0.032863,0.030199,0.033526,0.032000
AR_10day,0.085696,0.055324,0.020884,0.097194,0.149415,0.090926,0.097883,0.163953,0.128009,0.141726,0.173946
RW_10day,0.010100,0.010050,0.008832,0.008832,0.016583,0.029563,0.034843,0.037068,0.034205,0.036497,0.034612
AR_20day,0.102910,0.058071,0.024178,0.116265,0.174727,0.119093,0.087448,0.122223,0.128559,0.212375,0.248336
RW_20day,0.012903,0.012268,0.008916,0.008515,0.017514,0.026907,0.035612,0.039743,0.037573,0.040178,0.039373


### Result

According to the RMSE metrics, we can find that AR(1) model is not better than random walk since we take days with largest RMSE and predict the term structure around them.